# Practical 7: Word Embeddings
# 20BCE057 Devasy Patel

## Use Gensim to get Word embeddings for text classification




In [2]:
# word/document classification using word embeddings from gesim(Wor2Vec)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim

In [3]:
from sklearn.model_selection import train_test_split
# Use word embeddings for document/text classification.

df = pd.read_csv('train.csv')
df

,UserName,ScreenName,Location,TweetAt,text,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know its getting tough when @KameronWilds...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [4]:
# drop columns UserName, ScreenName, Location, TweetAt
df = df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
df

,text,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...
41152,Airline pilots offering to stock supermarket s...,Neutral
41153,Response to complaint not provided citing COVI...,Extremely Negative
41154,You know its getting tough when @KameronWilds...,Positive
41155,Is it wrong that the smell of hand sanitizer i...,Neutral


In [7]:
# now use the text column to convert to word embeddings
# use gensim to convert text to word embeddings
from gensim.models import Word2Vec
import gensim.downloader as api

# use pre-trained word embeddings for twitter
 
glove = api.load('glove-twitter-25')
glove

[==================================================] 100.0% 104.8/104.8MB downloaded


In [8]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout


In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(df['text'], df['Sentiment'], test_size=0.2, random_state=42)


In [26]:
def preprocess_text(text):
    text = text.lower()
    text = gensim.utils.simple_preprocess(text)
    return text
    
X_train = Xtrain.apply(preprocess_text)
X_test = Xtest.apply(preprocess_text)



In [ ]:
# download and save from https://nlp.stanford.edu/data/glove.twitter.27B.zip
# load the whole embedding into memory
import requests
import zipfile

url = 'https://nlp.stanford.edu/data/glove.twitter.27B.zip'
r = requests.get(url, allow_redirects=True)
open('glove.twitter.27B.zip', 'wb').write(r.content)


In [27]:
glove_model = KeyedVectors.load_word2vec_format('glove.twitter.27B.25d.txt', binary=False)


FileNotFoundError: [Errno 2] No such file or directory: 'glove.twitter.27B.25d.txt'

In [22]:
def word_vectors(words, model):
    vector = np.zeros(100)
    count = 0
    for word in words:
        try:
            vector += model[word]
            count += 1
        except:
            pass
    return vector

X_train_vectors = [word_vectors(words, model) for words in X_train]
X_test_vectors = [word_vectors(words, model) for words in X_test]


In [23]:
le = LabelEncoder()
y_train = le.fit_transform(ytrain)
y_test = le.transform(ytest)


In [24]:
model = Sequential()
model.add(Dense(256, input_dim=100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [25]:
model.fit(np.array(X_train_vectors), np.array(y_train), epochs=50, batch_size=32, validation_data=(np.array(X_test_vectors), np.array(y_test)))


Epoch 1/50
1029/1029 [==============================] - 12s 8ms/step - loss: -0.1630 - accuracy: 0.1607 - val_loss: -0.9865 - val_accuracy: 0.1616
Epoch 2/50
1029/1029 [==============================] - 11s 10ms/step - loss: -1.7020 - accuracy: 0.1608 - val_loss: -2.4330 - val_accuracy: 0.1616
Epoch 3/50
1029/1029 [==============================] - 9s 9ms/step - loss: -3.0755 - accuracy: 0.1608 - val_loss: -3.7774 - val_accuracy: 0.1616
Epoch 4/50
1029/1029 [==============================] - 5s 5ms/step - loss: -4.3906 - accuracy: 0.1608 - val_loss: -5.0972 - val_accuracy: 0.1616
Epoch 5/50
 548/1029 [==============>...............] - ETA: 2s - loss: -5.3767 - accuracy: 0.1580

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(np.array(X_test_vectors), np.array(y_test))
print('Test accuracy:', accuracy)
